<a href="https://colab.research.google.com/github/phanlinh0803/atom-assignments/blob/PhanLinh-assignment-04/ATOM_%7C_Phan_Linh_Home_Assignment_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install duckdb

     |████████████████████████████████| 8.5MB 20.2MB/s 


In [4]:
import requests # -> Để call API
import json # -> Xử lý file JSON
import pandas as pd # -> Thư viện xử lý dữ liệu dạng bảng
import re # -> Thư viện xử lý text: regular expressions
from datetime import datetime as dt # -> Thư viện xử lý dữ liệu thời gian
import duckdb # -> Thư viện "giả lập" xử lý dữ liệu bằng SQL
import numpy as np

**STEP 0: XEM LẠI DATA ĐÃ CÓ (SLACK API)**

Ở bước đầu tiên, ta xem lại tất cả các data của DATACracy Atom mà ra đã biết:

Data từ Slack API:
- Danh sách thành viên
- Danh sách các channels
- Lịch sử tin nhắn trên các channels
- Data do dự án tự collect - File CSV (trích xuất từ Google Spreadsheet):
- Danh sách thành viên được phân theo vị trí (mentors, learners, BTC)

In [7]:
## Load Token file 
## WARNING!! --- Put it in gitignore and DO NOT print out to notebook
with open('global_env.json', 'r') as j:
     json_data = json.load(j)

**0.1. Pull List of Members**

In [8]:
# 1. LIST OF SLACK MEMBERS 

## Pull list of member as JSON
## Gọi API từ Endpoints (Input - Token được đưa vào Headers)
## Challenge: Thử gọi API này bằng Postman
endpoint = "https://slack.com/api/users.list"
headers = {"Authorization": "Bearer {}".format(json_data['SLACK_BEARER_TOKEN'])}
response_json = requests.post(endpoint, headers=headers).json() 
user_dat = response_json['members']

## Convert to CSV
user_dict = {'user_id':[],'name':[],'display_name':[],'real_name':[],'title':[],'is_bot':[]}
for i in range(len(user_dat)):
  user_dict['user_id'].append(user_dat[i]['id'])
  user_dict['name'].append(user_dat[i]['name'])
  user_dict['display_name'].append(user_dat[i]['profile']['display_name'])
  user_dict['real_name'].append(user_dat[i]['profile']['real_name_normalized'])
  user_dict['title'].append(user_dat[i]['profile']['title'])
  user_dict['is_bot'].append(user_dat[i]['is_bot'])

user_df = pd.DataFrame(user_dict) 
user_df = user_df.replace('', np.nan) # -> replace khoảng trắng bằng giá trị NULL (nan)
user_df.head()

,user_id,name,display_name,real_name,title,is_bot
0,USLACKBOT,slackbot,Slackbot,Slackbot,NaN,False
1,U01AT4T75JB,loclexuan26392,Loc Le Xuan,Loc Le Xuan,NaN,False
2,U01AVDY7JET,locle.ds,Loc Le Xuan,Loc Le Xuan,NaN,False
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,False
4,U01C48T7S1J,huyenhoang.design,Thanh Huyen Hoang,Thanh Huyen Hoang,NaN,False


In [9]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       75 non-null     object
 1   name          75 non-null     object
 2   display_name  50 non-null     object
 3   real_name     75 non-null     object
 4   title         6 non-null      object
 5   is_bot        75 non-null     bool  
dtypes: bool(1), object(5)
memory usage: 3.1+ KB


**0.2. List of Channels**

In [16]:
# 2. LIST OF SLACK CHANNELS

endpoint2 = "https://slack.com/api/conversations.list"
data = {'types': 'public_channel,private_channel'} # -> CHECK: API Docs https://api.slack.com/methods/conversations.list/test
response_json = requests.post(endpoint2, headers=headers, data=data).json() 
channel_dat = response_json['channels']

channel_dict = {'channel_id':[], 'channel_name':[], 'is_channel':[],'creator':[],'created_at':[],'topics':[],'purpose':[],'num_members':[]}
for i in range(len(channel_dat)):
  channel_dict['channel_id'].append(channel_dat[i]['id'])
  channel_dict['channel_name'].append(channel_dat[i]['name'])
  channel_dict['is_channel'].append(channel_dat[i]['is_channel'])
  channel_dict['creator'].append(channel_dat[i]['creator'])
  channel_dict['created_at'].append(dt.fromtimestamp(float(channel_dat[i]['created'])))
  channel_dict['topics'].append(channel_dat[i]['topic']['value'])
  channel_dict['purpose'].append(channel_dat[i]['purpose']['value'])
  channel_dict['num_members'].append(channel_dat[i]['num_members'])

channel_df = pd.DataFrame(channel_dict) 
channel_df = channel_df.replace('', np.nan) # -> replace khoảng trắng bằng giá trị NULL (nan)
channel_df.head()

,channel_id,channel_name,is_channel,creator,created_at,topics,purpose,num_members
0,C01B4PVGLVB,general,True,U01BE2PR6LU,2020-09-23 10:25:03,NaN,This is the one channel that will always inclu...,65
1,C01BYH7JHB5,contents,True,U01BE2PR6LU,2020-10-05 07:28:09,NaN,NaN,12
2,C01CAMNCJJV,branding-design,True,U01AVDY7JET,2020-10-05 03:16:30,NaN,NaN,9
3,C01U6P7LZ8F,atom-assignment1,True,U01BE2PR6LU,2021-04-17 07:00:04,Assigment#1 - Python Basics &amp; Spreadsheet ...,Where learners submit assignments and give rev...,44
4,C01UL6K1C7L,atom-week1,True,U01BE2PR6LU,2021-04-16 17:44:32,TOPIC: Data Strategy. Metrics Design. Spreadsh...,TOPIC: Data Strategy. Metrics Design. Spreadsh...,43


In [17]:
channel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   channel_id    19 non-null     object        
 1   channel_name  19 non-null     object        
 2   is_channel    19 non-null     bool          
 3   creator       19 non-null     object        
 4   created_at    19 non-null     datetime64[ns]
 5   topics        6 non-null      object        
 6   purpose       13 non-null     object        
 7   num_members   19 non-null     int64         
dtypes: bool(1), datetime64[ns](1), int64(1), object(5)
memory usage: 1.2+ KB


**0.3. Message Data**

In [18]:
endpoint3 = "https://slack.com/api/conversations.history"

In [19]:
msg_dict = {'channel_id':[],'msg_id':[], 'msg_ts':[], 'user_id':[], 'latest_reply':[],'reply_user_count':[],'reply_users':[],'github_link':[]}
for channel_id, channel_name in zip(channel_df['channel_id'], channel_df['channel_name']):
  print('Channel ID: {} - Channel Name: {}'.format(channel_id, channel_name))
  try:
    data = {"channel": channel_id} 
    response_json = requests.post(endpoint3, data=data, headers=headers).json()
    msg_ls = response_json['messages']
    for i in range(len(msg_ls)):
      if 'client_msg_id' in msg_ls[i].keys():
        msg_dict['channel_id'].append(channel_id)
        msg_dict['msg_id'].append(msg_ls[i]['client_msg_id'])
        msg_dict['msg_ts'].append(dt.fromtimestamp(float(msg_ls[i]['ts'])))
        msg_dict['latest_reply'].append(dt.fromtimestamp(float(msg_ls[i]['latest_reply'] if 'latest_reply' in msg_ls[i].keys() else 0))) ## -> No reply: 1970-01-01
        msg_dict['user_id'].append(msg_ls[i]['user'])
        msg_dict['reply_user_count'].append(msg_ls[i]['reply_users_count'] if 'reply_users_count' in msg_ls[i].keys() else 0)
        msg_dict['reply_users'].append(msg_ls[i]['reply_users'] if 'reply_users' in msg_ls[i].keys() else 0) 
        ## -> Censor message contains tokens
        text = msg_ls[i]['text']
        github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text)
        msg_dict['github_link'].append(github_link[0] if len(github_link) > 0 else np.nan)
  except:
    print('====> '+ str(response_json))


Channel ID: C01B4PVGLVB - Channel Name: general
Channel ID: C01BYH7JHB5 - Channel Name: contents
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C01CAMNCJJV - Channel Name: branding-design
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C01U6P7LZ8F - Channel Name: atom-assignment1
Channel ID: C01UL6K1C7L - Channel Name: atom-week1
Channel ID: C01ULCHGN75 - Channel Name: atom-general
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C020VMT58JK - Channel Name: topics-data-analytics
Channel ID: C0213MNH9L6 - Channel Name: topics-python
Channel ID: C0213N56M2A - Channel Name: topics-materials
Channel ID: C021FSDN7LJ - Channel Name: atom-assignment2
Channel ID: C021KLB0DSB - Channel Name: discuss-group3
Channel ID: C021KLB90GP - Channel Name: discuss-group4
Channel ID: C02204B2CD6 - Channel Name: atom-week2
Channel ID: C0220KU9399 - Channel Name: discuss-group1
Channel ID: C0226D3LEQ4 - Channel Name: atom-week3
Channel ID: C0227A51SAY - Channel Name: atom-ass

In [20]:
msg_df = pd.DataFrame(msg_dict)
msg_df = msg_df.replace('', np.nan) # -> replace khoảng trắng bằng giá trị NULL (nan)
msg_df.tail()

,channel_id,msg_id,msg_ts,user_id,latest_reply,reply_user_count,reply_users,github_link
278,C022RRWQ6US,3db33d2a-5666-44cf-95e0-dd33aee4e63c,2021-05-24 11:49:29.003700,U01BE2PR6LU,1970-01-01 00:00:00.000000,0,0,NaN
279,C022RRWQ6US,8b4a2ebb-7882-402b-8339-a0b067ce5165,2021-05-23 07:58:23.003000,U01BE2PR6LU,1970-01-01 00:00:00.000000,0,0,NaN
280,C022RRWQ6US,231fad01-7083-4cd0-b6b6-e97bf54779e2,2021-05-22 23:23:13.001400,U01BE2PR6LU,1970-01-01 00:00:00.000000,0,0,https://github.com/anhdanggit/
281,C022Y1FUETE,ee582f3d-c339-4d06-a263-b71c6470e647,2021-05-24 11:49:57.001000,U01BE2PR6LU,1970-01-01 00:00:00.000000,0,0,NaN
282,C022Y1FUETE,38adf48f-21cc-437c-9ad3-31cd002bc4f8,2021-05-23 07:58:05.001500,U01BE2PR6LU,2021-05-23 16:15:45.002200,1,[U01VB632FPS],NaN


In [21]:
msg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283 entries, 0 to 282
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   channel_id        283 non-null    object        
 1   msg_id            283 non-null    object        
 2   msg_ts            283 non-null    datetime64[ns]
 3   user_id           283 non-null    object        
 4   latest_reply      283 non-null    datetime64[ns]
 5   reply_user_count  283 non-null    int64         
 6   reply_users       283 non-null    object        
 7   github_link       112 non-null    object        
dtypes: datetime64[ns](2), int64(1), object(5)
memory usage: 17.8+ KB


**0.4. DataCracy Info**

Data do dự án tự collect - File CSV (trích xuất từ Google Spreadsheet):
Danh sách thành viên được phân theo vị trí (mentors, learners, BTC)
Trong cùng folder Github assignment_4

In [22]:
dtc_groups = pd.read_csv('datacracy_groups.csv')
dtc_groups.head()

,name,DataCracy_role
0,slackbot,BTC
1,loclexuan26392,BTC
2,locle.ds,BTC
3,maianhdang.ftu,BTC
4,huyenhoang.design,BTC


**STEP 1: NHU CẦU & MỤC ĐÍCH**

> Đặt mình vào vị trí người chủ, bạn quan tâm đến điều gì?



Quan trọng nhất của mọi Data Solution, là bắt đầu từ nhu cầu, mục đích và câu hỏi lớn của Clients (người chủ).

Chính từ những câu hỏi lớn này, ta có thể khoanh vùng thông tin nào quan trọng, ta muốn đạt được điều gì?

**TODO#1: Requirements**

In [ ]:
''' 
REQUIREMENTS

- Mục đích của lớp học Atom là gì?
> Phổ cập kiến thức về khoa họcc dữ liệu đến cộng đồng, giúp các bạn học viên tham gia hiểu và có thể sử dụng được ngôn ngữ lập trình Python và các công cụ hỗ trợ cho quá trình phân tích dữ liệu từ đó biết cách ứng dụng những kiến thức đã học vào việc đưa ra các giải pháp về dữ liệu, hỗ trợ xử lý các vấn đề/ bài toán của công ty/doanh nghiệp.

- BTC sẽ quan tâm đến những chủ đề/quy trình gì để đạt được Mục Đích trong (1)?
> Thiết kế bài giảng sinh động, dễ hiểu nhằm truyền tải kiến thức nền tảng một cách dễ tiếp thu đến với các học viên.
> Đưa ra những ví dụ thực tế để cùng trao đổi, thảo luận để đào sâu hơn vào từng vấn đề từ đó mỗi học viên có thể rút ra được những bài học cho riêng mình.
> Bài tập thực hành hỗ trợ học viên trong việc tự khám phá và học hỏi thêm, bên cạnh những gì đã trao đổi trực tiếp trong lớp học.
> Cung cấp nguồn tài liệu học tập phù hợp để học viên có thể tra cứu thêm.
> Mức độ lan tỏa về dự án trong cộng đồng: Dự án nhận được sự hỗ trợ từ cá nhân/đơn vị/tổ chức/doanh nghiệp. Học viên chia sẻ những điều học được từ lớp học đến nhiều người hơn. 

- Làm sao để đo lường các điểm trong (2)?
> Khảo sát về mức độ hiểu và tiếp thu kiến thức của học viên sau sau mỗi buổi học.
> Đánh giá về mức độ sôi nổi của lớp học khi thảo luận trên channel chung.
(? % học viên tham gia thảo luận/đóng góp ý kiến, có bao nhiêu chủ đề được nêu lên? bao nhiêu chủ đề được mổ xẻ và giải quyết cùng nhau?...)
> Mức độ hoàn thành của mỗi học viên đối với bài tập, số lượng học viên hoàn thành bài tập theo yêu cầu?...
> Bao nhiêu lượt tương tác/bài viết nhắc đến dự án được chia sẻ? Bao nhiêu người biết thêm về dự án sau khi lớp học kết thúc?

- Dựa vào các data đã có như liệt kê trong STEP 0: Chỉ dùng những data sẵn có, ta có thể đo lường và thiết kế những metrics nào bạn đã liệt kê trong (3)?
> Mức độ hiểu: Đánh giá trên thang điểm 5.
> Mức độ hài lòng về sự hỗ trợ của Coach/menter: Đánh giá trên thang điểm 5.
> Số lượng học viên tham gia các buổi học.
> Số lượng học viên hoàn thành bài tập.
> Số lượng bài tập hoàn thành/học viên.
> Điểm đánh giá chất lượng học viên.
> Số lượng chủ đề được nêu ra cùng tham gia thảo luận.
> TAT: số lượng bài viết chia sẻ/nhắc đến dự án hoặc lượt theo dõi/tương tác/like/share trên fanpage của dự án.

- Tham khảo Slack API và hình dung về các thông tin DataCracy có khả năng thu thập, bạn sẽ đề xuất DataCracy thu thập thêm những thông tin gì?
> Số lượt tương tác của các học viên.
> Số lượng học viên tham gia trao đổi trong mỗi channel.
> Số lượng bài assignment đã submit.
> Thời gian nộp bài của các học viên.


**STEP 2: TỔ CHỨC THÔNG TIN**

In [135]:
user_df.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       75 non-null     object
 1   name          75 non-null     object
 2   display_name  50 non-null     object
 3   real_name     75 non-null     object
 4   title         6 non-null      object
 5   is_bot        75 non-null     bool  
dtypes: bool(1), object(5)
memory usage: 3.1+ KB


In [132]:
channel_df.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   channel_id    19 non-null     object        
 1   channel_name  19 non-null     object        
 2   is_channel    19 non-null     bool          
 3   creator       19 non-null     object        
 4   created_at    19 non-null     datetime64[ns]
 5   topics        6 non-null      object        
 6   purpose       13 non-null     object        
 7   num_members   19 non-null     int64         
dtypes: bool(1), datetime64[ns](1), int64(1), object(5)
memory usage: 1.2+ KB


In [133]:
msg_df.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283 entries, 0 to 282
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   channel_id        283 non-null    object        
 1   msg_id            283 non-null    object        
 2   msg_ts            283 non-null    datetime64[ns]
 3   user_id           283 non-null    object        
 4   latest_reply      283 non-null    datetime64[ns]
 5   reply_user_count  283 non-null    int64         
 6   reply_users       283 non-null    object        
 7   github_link       112 non-null    object        
dtypes: datetime64[ns](2), int64(1), object(5)
memory usage: 17.8+ KB


In [134]:
dtc_groups.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            65 non-null     object
 1   DataCracy_role  59 non-null     object
dtypes: object(2)
memory usage: 1.1+ KB


**TODO#2: List Down**

In [54]:
user_df.describe(include = 'all')

,user_id,name,display_name,real_name,title,is_bot
count,75,75,50,75,6,75
unique,75,75,49,73,5,2
top,U01UJ9PA0MB,78,Loc Le Xuan,Loc Le Xuan,Learner,False
freq,1,1,2,2,2,65


In [ ]:
 Dtype obj: object -> string, bool, int, float
# NULL - Nếu có null count < số entries

'''
user_df
-
user_id string 
name string
display_name NULL string
real_name string
title NULL string
is_bot bool

-
channel_id string 
channel_name string
is_channel bool
creator string 
create_at datetime
topics NULL string
purpose NULL string
num_members int

msg_df
-
channel_id string 
msg_id string 
msg_ts string
user_id string 
latest_reply datetime
reply_user_count int
reply_users string
github_link NULL string

dtc_groups
-
name string
DataCracy_role NULL string

'''

**STEP 3: NGUYÊN TẮC CẦN ÁP DỤNG**

**TODO#3: Rules & Logics**

In [ ]:
''' 
RULES khi giao tiếp trên Slack
---
1.Quy tắc tạo tài khoản cho user:
    - Ghi rõ các thông tin họ & tên
    - Ghi rõ title/vai trò và nhóm của mình
    Cú pháp: username_tilte_group . VD: phanlinh_learner_r3
    
2. Quy tắc tạo channel:
    - Ghi rõ tên, mục đích, topic thảo luận của từng channel
    Cú pháp: #Atom_assignment_04, #R3_discussion

3. Quy tắc gửi tin nhắn:
    - Nêu rõ mong muốn/câu hỏi/ý kiến khi gửi tin nhắn.
    - Có thể thêm #hashtag vào đầu tin để xác định chủ đề.
    VD: #assignment04 #issue
    
4. Quy tắc nộp bài:
    - Gửi bài về đúng channel.  
    - Chèn hashtag tương ứng với mỗi link bài tập đính kèm.
    - Note lưu ý riêng của bản thân (nếu có)       

5. Quy tắc review bài tập
    - Gắn label trước khi review.
    - Review trực tiếp bằng cách trả lời tin nhắn của người gửi.
    - Đổi label sau khi đã review xong.
''' 

In [ ]:
''' 
LOGICS
---
1. #user:
- hai users trùng tên -> phân biệt bằng title/group hoặc user_id nên cần cập nhật thông tin title hoặc group.
2. #channel:
- topic phải được gửi đúng channel, phân biệt bằng hashtag #assignment
- ngày gửi assignment < ngày tạo channel 
3. #msg:
- lasted reply < msg_ts
- khi gửi tin nhắn vào channel chung, phải có hashtag ghi rõ topic, thời gian nộp bài.
- 
'''

**STEP 4: TỔ CHỨC BẢNG - PRIMARY KEYS**

**TODO#4: Tables & PK**

In [ ]:
#Nhìn lại diagram của TODO#2 trong Quick DBD Diagram: Có bảng nào bạn nghĩ nên gộp lại, hay tách ra ?

#Tìm Primary Key (PK): Unique cho từng dòng và Không NULL

In [57]:
## Hints: nunique() để check số giá trị unique của từng trường
user_df.nunique()

user_id         75
name            75
display_name    49
real_name       73
title            5
is_bot           2
dtype: int64

In [36]:
channel_df.nunique()

channel_id      19
channel_name    19
is_channel       1
creator          2
created_at      19
topics           6
purpose         12
num_members     15
dtype: int64

In [137]:
msg_df.head ()

,channel_id,msg_id,msg_ts,user_id,latest_reply,reply_user_count,reply_users,github_link
0,C01B4PVGLVB,0b9a47d9-900e-4dcf-af8f-8897e6fe6fdd,2021-05-28 15:35:27.004100,U01BE2PR6LU,1970-01-01 00:00:00.000000,0,0,NaN
1,C01B4PVGLVB,DEBF1A3C-17AA-4D53-AC27-1FE0884243DB,2021-05-28 13:50:27.000800,U01JQ5U1RPV,2021-05-28 13:51:51.001200,1,[U01UQGZHSTZ],NaN
2,C01B4PVGLVB,48ba219d-fda8-40f2-be65-00735c058071,2021-05-25 04:59:27.000600,U01BE2PR6LU,2021-05-25 06:04:44.006500,4,"[U01UJ9PRT5K, U01UTGRMGEQ, U01VB632FPS, U01UMB...",NaN
3,C01B4PVGLVB,89919b55-bfe9-4c15-bf45-3021048c7c28,2021-05-24 11:49:08.061100,U01BE2PR6LU,1970-01-01 00:00:00.000000,0,0,NaN
4,C01B4PVGLVB,18467584-5810-4bb9-80fe-4b92af9b28d0,2021-05-23 16:26:04.057000,U01UTGT8CVA,2021-05-24 08:36:52.058100,2,"[U01VB632FPS, U01UTGT8CVA]",NaN


In [37]:
dtc_groups.nunique()

name              65
DataCracy_role    10
dtype: int64

In [ ]:
## Copy lại phần text của TODO#2
## Đặt PK bên cạnh col bạn chọn làm PK
''' 
# Dtype obj: object -> string, bool, int, float
# NULL - Nếu có null count < số entries

user_df
-
user_id string 
name string
display_name NULL string
real_name string
title NULL string
is_bot bool

-
channel_id string 
channel_name string
is_channel bool
creator string 
create_at datetime
topics NULL string
purpose NULL string
num_members int

msg_df
-
channel_id string 
msg_id string 
msg_ts string
user_id string 
latest_reply datetime
reply_user_count int
reply_users string
github_link NULL string

dtc_groups
-
name string
DataCracy_role NULL string

'''

**STEP 5: MỐI QUAN HỆ GIỮA CÁC BẢNG**

**TODO#5: FK & Mapping**

**STEP 6: DIAGRAM & TEST**

In [49]:
## Đoạn code dưới join 2 bảng user_df vằ dtc_groups bằng key name => Tạo thành bảng members_df
SQL_dim_members = '''
-- Để comment trong SQL dùng -- tương đường với ## trong Python
CREATE TABLE members_df AS
SELECT 
  t1.*,
  t2.DataCracy_role
FROM user_df AS t1
JOIN dtc_groups AS t2
ON t1.name = t2.name
'''

In [50]:
con = duckdb.connect(database=':memory:', read_only=False) # -> Tạo DB connection
# create a table
con.execute(SQL_dim_members) # -> Chạy đoạn lệch SQL
con.execute("SELECT * FROM members_df LIMIT 10").fetch_df() # -> In 10 dòng đầu tiên của bảng members_df ra Dataframe

,user_id,name,display_name,real_name,title,is_bot,DataCracy_role
0,USLACKBOT,slackbot,Slackbot,Slackbot,NaN,False,BTC
1,U01AT4T75JB,loclexuan26392,Loc Le Xuan,Loc Le Xuan,NaN,False,BTC
2,U01AVDY7JET,locle.ds,Loc Le Xuan,Loc Le Xuan,NaN,False,BTC
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,False,BTC
4,U01C48T7S1J,huyenhoang.design,Thanh Huyen Hoang,Thanh Huyen Hoang,NaN,False,BTC
5,U01CMTYB7EU,dr.tringuyenzmc,NaN,Dr Tringuyenzmc,NaN,False,BTC
6,U01GQR2FE93,khanguyenwk,NaN,Kha Nguyen,NaN,False,Volunteer
7,U01HARCHNKU,linhphan,NaN,Phan Linh,NaN,False,BTC
8,U01JQ5TUP0B,tt.thuchuyen,NaN,Tt Thuchuyen,NaN,False,BTC
9,U01JQ5U1RPV,thuongthuong.dv,NaN,Thuong Do,NaN,False,BTC


In [51]:
con.execute("SELECT * FROM channel_df LIMIT 5").fetch_df() # -> In 5 dòng đầu tiên của bảng channel_df ra Dataframe

,channel_id,channel_name,is_channel,creator,created_at,topics,purpose,num_members
0,C01B4PVGLVB,general,True,U01BE2PR6LU,2020-09-23 10:25:03,NaN,This is the one channel that will always inclu...,65
1,C01BYH7JHB5,contents,True,U01BE2PR6LU,2020-10-05 07:28:09,NaN,NaN,12
2,C01CAMNCJJV,branding-design,True,U01AVDY7JET,2020-10-05 03:16:30,NaN,NaN,9
3,C01U6P7LZ8F,atom-assignment1,True,U01BE2PR6LU,2021-04-17 07:00:04,Assigment#1 - Python Basics &amp; Spreadsheet ...,Where learners submit assignments and give rev...,44
4,C01UL6K1C7L,atom-week1,True,U01BE2PR6LU,2021-04-16 17:44:32,TOPIC: Data Strategy. Metrics Design. Spreadsh...,TOPIC: Data Strategy. Metrics Design. Spreadsh...,43


In [52]:
con.execute("SELECT * FROM msg_df LIMIT 5").fetch_df() # -> In 5 dòng đầu tiên của bảng msg_df ra Dataframe

,channel_id,msg_id,msg_ts,user_id,latest_reply,reply_user_count,reply_users,github_link
0,C01B4PVGLVB,0b9a47d9-900e-4dcf-af8f-8897e6fe6fdd,2021-05-28 15:35:27.004100,U01BE2PR6LU,1970-01-01 00:00:00.000000,0,NaN,NaN
1,C01B4PVGLVB,DEBF1A3C-17AA-4D53-AC27-1FE0884243DB,2021-05-28 13:50:27.000800,U01JQ5U1RPV,2021-05-28 13:51:51.001200,1,NaN,NaN
2,C01B4PVGLVB,48ba219d-fda8-40f2-be65-00735c058071,2021-05-25 04:59:27.000600,U01BE2PR6LU,2021-05-25 06:04:44.006500,4,NaN,NaN
3,C01B4PVGLVB,89919b55-bfe9-4c15-bf45-3021048c7c28,2021-05-24 11:49:08.061100,U01BE2PR6LU,1970-01-01 00:00:00.000000,0,NaN,NaN
4,C01B4PVGLVB,18467584-5810-4bb9-80fe-4b92af9b28d0,2021-05-23 16:26:04.057000,U01UTGT8CVA,2021-05-24 08:36:52.058100,2,NaN,NaN


In [53]:
## Đoạn code dưới: Theo từng channel, count số message (phải join với bảng channel_df để lấy tên channel)
SQL = '''
SELECT 
  t2.channel_name,
  COUNT(DISTINCT t1.msg_id) AS msg_cnt
FROM msg_df AS t1
JOIN channel_df AS t2
ON t1.channel_id = t2.channel_id
GROUP BY t2.channel_name 
ORDER BY COUNT(DISTINCT t1.msg_id) DESC
'''
con.execute(SQL).fetch_df()

,channel_name,msg_cnt
0,general,44
1,atom-assignment1,42
2,atom-assignment2,42
3,atom-assignment3,38
4,discuss-group1,32
5,atom-assignment4,23
6,discuss-group3,18
7,topics-python,13
8,discuss-group4,12
9,atom-week1,6


In [118]:
## Đoạn code dưới: Lấy top 3 mentors post nhiều message nhất trong discuss-group của các nhóm
SQL = '''
WITH msg_cnt AS ( ------------- > (1) Chain SQL: tạo bảng tạm thời msg_cnt: count số msg theo user, theo channel
  SELECT 
    user_id,
    channel_id,
    COUNT(msg_id) AS msg_cnt
  FROM msg_df 
  GROUP BY 1, 2
)
SELECT
  t2.real_name,
  t3.channel_name,
  t1.msg_cnt,
  t2.DataCracy_role
FROM msg_cnt AS t1
JOIN members_df AS t2 --- > (2) Join msg_count với members_df để lấy tên và role
ON t1.user_id = t2.user_id
JOIN channel_df AS t3 --> (3) Join với channel_df để lấy tên channel
ON t1.channel_id = t3.channel_id
WHERE t2.DataCracy_role LIKE 'Mentor%' ---- > (4) Filter Mentors
AND t3.channel_name LIKE 'discuss-group%' ---- > (5) Filter channel discuss theo các group
ORDER BY t1.msg_cnt DESC ---- > (6) Sắp xếp theo số msg từ cao xuống thấp
LIMIT 3 ---> (7) Lấy top 3
'''
con.execute(SQL).fetch_df()

,real_name,channel_name,msg_cnt,DataCracy_role
0,Phu NDT,discuss-group1,11,Mentor_Gr1
1,Nguyễn Quang Long,discuss-group3,10,Mentor_Gr3
2,Thuy Nguyen,discuss-group4,4,Mentor_Gr4


**TODO#6: SQL**

In [140]:
#Learners group nào hoạt động tích cực nhất trên Slack nhất?
SQL_1_1 = '''
WITH msg_cnt AS (---> (1) Chain SQL: tạo bảng tạm thời msg_cnt: count số msg theo user, theo channel
  SELECT user_id, COUNT(msg_id) AS msg_cnt FROM msg_df GROUP BY 1)
SELECT t2.real_name, t1.msg_cnt, t2.DataCracy_role FROM msg_cnt AS t1
JOIN members_df AS t2 --- > (2) Join msg_count với members_df để lấy tên và role
ON t1.user_id = t2.user_id
WHERE t2.DataCracy_role LIKE 'Learner%' --- > (3) Lọc Learners
ORDER BY t1.msg_cnt DESC --- > (4) Sắp xếp theo thứ tự msg
LIMIT 3 ---> (5) Lấy top 3
'''
con.execute(SQL_1_1).fetch_df()

,real_name,msg_cnt,DataCracy_role
0,Việt Hồng Chu,15,Learner_Gr1
1,Hanh Nguyen,11,Learner_Gr4
2,Quan Nguyễn,10,Learner_Gr4


In [145]:
#Learners nào nộp bài sớm nhất trong Assignment 1?
SQL_2_1 = '''
SELECT ms.user_id, mb.real_name, DataCracy_role, channel_name, msg_ts datetime, github_link

FROM members_df mb JOIN msg_df ms on mb.user_id = ms.user_id
JOIN channel_df ch ON ch.channel_id=ms.channel_id

WHERE ms.msg_ts = (SELECT Min(ms.msg_ts) 
                    FROM msg_df ms join channel_df ch ON ms.channel_id = ch.channel_id 
                        JOIN members_df mb ON mb.user_id = ms.user_id
                    WHERE ch.channel_name = 'atom-assignment1' AND ms.github_link != 'NaN' 
                        AND mb.DataCracy_role like 'Learner%' )
'''
con.execute(SQL_2_1).fetch_df()

,user_id,real_name,DataCracy_role,channel_name,datetime,github_link
0,U01UMJ5AQ0K,Hưng Dương Phan Nam,Learner_Gr1,atom-assignment1,2021-04-21 06:00:34.000800,https://github.com/duongphannamhung/


In [146]:
#Learners nào nộp bài sớm nhất trong Assignment 2?
SQL_2_2 = '''
SELECT ms.user_id, mb.real_name, DataCracy_role, channel_name, msg_ts datetime, github_link

FROM members_df mb JOIN msg_df ms ON mb.user_id = ms.user_id
JOIN channel_df ch ON ch.channel_id=ms.channel_id

WHERE ms.msg_ts = (SELECT Min(ms.msg_ts) 
                    FROM msg_df ms join channel_df ch ON ms.channel_id = ch.channel_id 
                        join members_df mb ON mb.user_id = ms.user_id
                    WHERE ch.channel_name = 'atom-assignment2' AND ms.github_link != 'NaN' 
                        AND mb.DataCracy_role like 'Learner%' )
'''
con.execute(SQL_2_2).fetch_df()

,user_id,real_name,DataCracy_role,channel_name,datetime,github_link
0,U01UQGZHSTZ,Toan Tran,Learner_Gr2,atom-assignment2,2021-05-12 15:50:48.004,https://github.com/toantran2206/


In [147]:
#Learners nào nộp bài sớm nhất trong Assignment 3?
SQL_2_3 = '''
SELECT ms.user_id, mb.real_name, DataCracy_role, channel_name, msg_ts datetime, github_link

FROM members_df mb JOIN msg_df ms ON mb.user_id = ms.user_id
JOIN channel_df ch ON ch.channel_id=ms.channel_id

WHERE ms.msg_ts = (SELECT MIN(ms.msg_ts) 
                    FROM msg_df ms join channel_df ch ON ms.channel_id = ch.channel_id 
                        join members_df mb ON mb.user_id = ms.user_id
                    WHERE ch.channel_name = 'atom-assignment3' AND ms.github_link != 'NaN' 
                        AND mb.DataCracy_role like 'Learner%' )
'''
con.execute(SQL_2_3).fetch_df()

,user_id,real_name,DataCracy_role,channel_name,datetime,github_link
0,U01UTGRMGEQ,Hanh Nguyen,Learner_Gr4,atom-assignment3,2021-05-16 12:08:08.018900,https://github.com/nguyenhonghanh2511/


In [144]:
#Learner nào chưa nộp bài Assignment 3?
SQL_1 = '''
SELECT mb.user_id, mb.real_name, DataCracy_role
FROM members_df mb
WHERE mb.DataCracy_role like'Learner%' and 
mb.user_id not in (select t1.user_id from msg_df t1 join channel_df t2 on t1.channel_id = t2.channel_id 
JOIN members_df t3 on t3.user_id = t1.user_id
WHERE t2.channel_name = 'atom-assignment3' and t1.github_link != 'NaN'
                                            and t3.DataCracy_role like'Learner%')
'''
con.execute(SQL_1).fetch_df()

,user_id,real_name,DataCracy_role
0,U01U6JM6LEB,Viet Hoang,Learner_Gr4
1,U01U8CAMGB1,Ngoc Huyen,Learner_Gr1
2,U01UEJG8KHU,Trang Nguyễn,Learner_Gr3
3,U01UJ9M5TU5,Nguyen Thi Kim Minh,Learner_Gr2
4,U01UMBXJLHG,Nguyen Linh,Learner_Gr1
5,U01UMC08KL2,Minh-Thu Nguyen Hoang,Learner_Gr4
6,U01UMC0N5U2,Việt Hồng Chu,Learner_Gr1
7,U01UTGRP0HJ,Tran Pham,Learner_Gr3
8,U01UTGRQS9J,Linh Vu,Learner_Gr4
9,U01VB63LRNC,anh thu nguyen huynh,Learner_Gr4


In [123]:
#Learner nào chưa nộp bất kỳ 1 assignment nào?

SQL_1 = '''
SELECT mb.user_id, mb.real_name, DataCracy_role
FROM members_df mb
WHERE mb.DataCracy_role like'Learner%' and 
mb.user_id not in (select t1.user_id from msg_df t1 join channel_df t2 on t1.channel_id = t2.channel_id 
JOIN members_df t3 on t3.user_id = t1.user_id
WHERE t2.channel_name like 'atom-assignment_' and t1.github_link != 'NaN'
                                            and t3.DataCracy_role like'Learner%')
'''
con.execute(SQL_1).fetch_df()

,user_id,real_name,DataCracy_role
0,U01U8CAMGB1,Ngoc Huyen,Learner_Gr1
1,U01UMBXJLHG,Nguyen Linh,Learner_Gr1
2,U01VBFY2E59,Gia Hoang Tran,Learner_Gr1
3,U0233KHLM2M,Vuthanhdat Contact,Learner_Gr4


In [149]:
#Tỷ lệ % Learner đã nộp assignment 1, 2, 3, 4?
SQL_1 = '''
SELECT t1.channel_name, format(t1.submited_cnt * 100/ t.learner_cnt,'P') as 'Tỉ lệ nộp bài (%)'
FROM (select count(user_id) as 'learner_cnt'
FROM members_df WHERE DataCracy_role like 'Learner%' ) t, 
    (select count(distinct t2.user_id) as 'submited_cnt', t3.channel_name
FROM msg_df t2 JOIN channel_df t3 on t2.channel_id = t3.channel_id JOIN members_df t4 on t4.user_id = t2.user_id
WHERE channel_name like'atom-assignment_' and t4.DataCracy_role like 'Learner%' and t2.github_link != 'NaN'
        GROUP BY t3.channel_name) t1
'''
con.execute(SQL_1).fetch_df()

,channel_name,Tỉ lệ nộp bài (%)
0,atom-assignment1,71
1,atom-assignment2,79
2,atom-assignment3,66
3,atom-assignment4,41


In [126]:
#Tỷ lệ % Learner đã submit bài và dc review trong assignment 1, 2, 3, 4?
SQL_1 = '''
SELECT t1.channel_name, format(t1.submited_cnt * 100/ t.learner_cnt, 'P') as 'Tỉ lệ bài được review (%)'
FROM (select count(user_id) as 'learner_cnt'
FROM members_df WHERE DataCracy_role like 'Learner%' ) t, 
    (select count(distinct t2.user_id) as 'submited_cnt', t3.channel_name
FROM msg_df t2 join channel_df t3 on t2.channel_id = t3.channel_id
JOIN members_df t4 on t4.user_id = t2.user_id
WHERE channel_name like'atom-assignment_' and t4.DataCracy_role like 'Learner%' and t2.github_link != 'NaN'
        and t2.reply_user_count > 0
        GROUP BY t3.channel_name) t1
'''
con.execute(SQL_1).fetch_df()

,channel_name,Tỉ lệ bài được review (%)
0,atom-assignment1,61
1,atom-assignment2,74
2,atom-assignment3,38
3,atom-assignment4,2


In [111]:
#Learners theo Group nào có tỷ lệ % hoàn thành bài tập cao nhất?
SQL_1 = '''
WITH report_byGroup AS(
SELECT t1.channel_name,t1.DataCracy_role, (t1.submited_cnt * 100/ t.learner_cnt) as 'Ti_le_nop'
FROM (select count(user_id) as 'learner_cnt', DataCracy_role
FROM members_df WHERE DataCracy_role like 'Learner%'
        GROUP BY DataCracy_role ) t join
    (select count(distinct t2.user_id) as 'submited_cnt', t3.channel_name,t4.DataCracy_role 
FROM msg_df t2 join channel_df t3 on t2.channel_id = t3.channel_id
JOIN members_df t4 on t4.user_id = t2.user_id
WHERE channel_name like'atom-assignment_' and t4.DataCracy_role like 'Learner%' and t2.github_link != 'NaN'
        GROUP BY t3.channel_name,t4.DataCracy_role  ) t1 
    on t.DataCracy_role = t1.DataCracy_role)
SELECT re.DataCracy_role, round(avg(re.Ti_le_nop),2) as 'Tỉ lệ hoàn thành bài tập (%)'
FROM report_byGroup re GROUP BY re.DataCracy_role
having  avg(re.Ti_le_nop) >= all(select avg(Ti_le_nop)
FROM report_byGroup GROUP BY DataCracy_role)
'''
con.execute(SQL_1).fetch_df()

,DataCracy_role,Tỉ lệ hoàn thành bài tập (%)
0,Learner_Gr2,80.0


**Pandas vs. SQL**

In [105]:
## Ví dụ: Đoạn code (Pandas) sau làm
## 1. Group by channel_id
## 2. Count các message
## 3. Sắp xếp theo thứ tự số message từ cao xuống thấp (ascending=False)
## 4. Filter lấy Top 5
msg_df.groupby('channel_id')['msg_id'].count().sort_values(ascending=False).head(5)

channel_id
C01B4PVGLVB    44
C021FSDN7LJ    42
C01U6P7LZ8F    42
C0227A51SAY    38
C0220KU9399    32
Name: msg_id, dtype: int64

In [106]:
## Kết quả tương ứng bằng SQL
SQL = '''
  SELECT 
    channel_id,
    COUNT(msg_id) AS msg_cnt
  FROM msg_df 
  GROUP BY 1
  ORDER BY COUNT(msg_id) DESC
  LIMIT 5
'''
con.execute(SQL).fetch_df()

,channel_id,msg_cnt
0,C01B4PVGLVB,44
1,C01U6P7LZ8F,42
2,C021FSDN7LJ,42
3,C0227A51SAY,38
4,C0220KU9399,32
